### Post Processing of the Heat Flux Vector Fields of the 2D Diffusion Equation

In [2]:
import matplotlib.pyplot as plt
import numpy as np

### Read the Data

In [3]:
def read_tec(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Find VARIABLES line
    for i, line in enumerate(lines):
        if line.strip().startswith("VARIABLES"):
            variables = [v.strip().strip('"') for v in line.split("=")[1].split(",")]
            data_start = i + 1
            break

    while not lines[data_start].strip()[0].isdigit() and not lines[data_start].strip().startswith('-'):
            data_start += 1

    import numpy as np
    data = np.loadtxt(lines[data_start:])
    return variables, data

def get_grid_shape(filename):
    nx = ny = None
    with open(filename, 'r') as f:
        for line in f:
            if line.strip().startswith("ZONE"):
                # Look for I= and J= in the line
                parts = line.replace(",", " ").split()
                for p in parts:
                    if p.startswith("I="):
                        nx = int(p.split("=")[1])
                    elif p.startswith("J="):
                        ny = int(p.split("=")[1])
                break
    return nx, ny


### Grid Independence

In [17]:
import numpy as np
from scipy.interpolate import griddata
import glob

def extract_temperature_and_coords(filename):
    variables, data = read_tec(filename)
    nx, ny = get_grid_shape(filename)
    if nx is None or ny is None:
        raise ValueError(f"Could not find grid shape (I, J) in file {filename}")

    # Identify columns for X, Y, Temperature
    try:
        x_idx = variables.index("X")
        y_idx = variables.index("Y")
        T_idx = variables.index("T")
    except ValueError as e:
        raise ValueError("X, Y or Temperature variable not found in variables list.")

    # Reshape data columns into 2D arrays (assuming data ordered accordingly)
    X = data[:, x_idx].reshape((ny, nx))
    Y = data[:, y_idx].reshape((ny, nx))
    T = data[:, T_idx].reshape((ny, nx))

    return X, Y, T

def compute_l2_norm(T1, T2):
    diff = T1 - T2
    return np.sqrt(np.sum(diff**2))

def main():
    files = sorted(glob.glob('output_*.tec'))
    print(f"Found files: {files}")

    for i in range(len(files) - 1):
        print(f"\nComparing {files[i]} and {files[i+1]}:")

        # Extract data
        X1, Y1, T1 = extract_temperature_and_coords(files[i])
        X2, Y2, T2 = extract_temperature_and_coords(files[i+1])

        # Flatten coordinates and temperatures for interpolation
        points1 = np.column_stack((X1.ravel(), Y1.ravel()))
        points2 = np.column_stack((X2.ravel(), Y2.ravel()))
        T1_flat = T1.ravel()
        T2_flat = T2.ravel()

        # Interpolate T1 (coarse) onto points2 (fine grid)
        T1_interp = griddata(points1, T1_flat, points2, method='linear')

        # Fill any NaNs resulted from interpolation outside convex hull
        nan_mask = np.isnan(T1_interp)
        if np.any(nan_mask):
            T1_interp[nan_mask] = griddata(points1, T1_flat, points2[nan_mask], method='nearest')

        # Compute L2 norm difference on fine grid
        l2_norm = compute_l2_norm(T1_interp, T2_flat)
        norm_T2 = np.sqrt(np.sum(T2_flat**2))
        rel_error = l2_norm / norm_T2

        print(f"L2 norm of temperature difference: {l2_norm:.6f}")
        print(f"Relative error: {rel_error:.6f}")

if __name__ == "__main__":
    main()


Found files: ['output_1.tec', 'output_2.tec', 'output_3.tec', 'output_4.tec', 'output_5.tec', 'output_6.tec']

Comparing output_1.tec and output_2.tec:
L2 norm of temperature difference: 46.457990
Relative error: 0.262675

Comparing output_2.tec and output_3.tec:
L2 norm of temperature difference: 60.757297
Relative error: 0.193587

Comparing output_3.tec and output_4.tec:
L2 norm of temperature difference: 356.475877
Relative error: 0.591342

Comparing output_4.tec and output_5.tec:
L2 norm of temperature difference: 2831.803373
Relative error: 0.834313

Comparing output_5.tec and output_6.tec:
L2 norm of temperature difference: 22503.284112
Relative error: 0.775931


In [16]:
import numpy as np
import glob

def extract_flux_and_coords(filename):
    variables, data = read_tec(filename)
    nx, ny = get_grid_shape(filename)
    if nx is None or ny is None:
        raise ValueError(f"Could not find grid shape (I, J) in file {filename}")

    try:
        x_idx = variables.index("X")
        y_idx = variables.index("Y")
        qx_idx = variables.index("QX")
        qy_idx = variables.index("QY")
    except ValueError:
        raise ValueError("X, Y, qx or qy variable not found in variables list.")

    X = data[:, x_idx].reshape((ny, nx))
    Y = data[:, y_idx].reshape((ny, nx))
    qx = data[:, qx_idx].reshape((ny, nx))
    qy = data[:, qy_idx].reshape((ny, nx))

    return X, Y, qx, qy

def compute_cell_areas(X, Y):
    ny, nx = X.shape
    nx_cells = nx - 1
    ny_cells = ny - 1

    areas = np.zeros((ny_cells, nx_cells))

    for j in range(ny_cells):
        for i in range(nx_cells):
            x0, y0 = X[j, i], Y[j, i]
            x1, y1 = X[j, i+1], Y[j, i+1]
            x2, y2 = X[j+1, i+1], Y[j+1, i+1]
            x3, y3 = X[j+1, i], Y[j+1, i]

            area = 0.5 * abs(x0*y1 - y0*x1 + x1*y2 - y1*x2 + x2*y3 - y2*x3 + x3*y0 - y3*x0)
            areas[j, i] = area

    return areas

def compute_volume_averaged_heat_flux(filename):
    X, Y, qx, qy = extract_flux_and_coords(filename)
    areas = compute_cell_areas(X, Y)
    qx_cells = 0.25 * (qx[:-1, :-1] + qx[:-1, 1:] + qx[1:, :-1] + qx[1:, 1:])
    qy_cells = 0.25 * (qy[:-1, :-1] + qy[:-1, 1:] + qy[1:, :-1] + qy[1:, 1:])

    total_area = np.sum(areas)
    qx_avg = np.sum(qx_cells * areas) / total_area
    qy_avg = np.sum(qy_cells * areas) / total_area

    return qx_avg, qy_avg

def relative_error(val_old, val_new):
    return abs(val_new - val_old) / (abs(val_new) + 1e-15)  # Avoid divide by zero

def main():
    files = sorted(glob.glob('output_*.tec'))
    print(f"Found files: {files}")

    avg_fluxes = []

    for f in files:
        qx_avg, qy_avg = compute_volume_averaged_heat_flux(f)
        avg_fluxes.append((qx_avg, qy_avg))
        print(f"{f}: qx_avg = {qx_avg:.6f}, qy_avg = {qy_avg:.6f}")

    print("\nComparing consecutive outputs:")

    for i in range(len(avg_fluxes) - 1):
        qx1, qy1 = avg_fluxes[i]
        qx2, qy2 = avg_fluxes[i+1]

        diff_qx = qx2 - qx1
        diff_qy = qy2 - qy1
        rel_qx = relative_error(qx1, qx2)
        rel_qy = relative_error(qy1, qy2)

        print(f"Between {files[i]} and {files[i+1]}:")
        print(f"  Δqx = {diff_qx:.6f}, relative change = {rel_qx*100:.3f}%")
        print(f"  Δqy = {diff_qy:.6f}, relative change = {rel_qy*100:.3f}%")

if __name__ == "__main__":
    main()


Found files: ['output_1.tec', 'output_2.tec', 'output_3.tec', 'output_4.tec', 'output_5.tec', 'output_6.tec']
output_1.tec: qx_avg = -584.338833, qy_avg = 107.751538
output_2.tec: qx_avg = -627.693496, qy_avg = 91.598172
output_3.tec: qx_avg = -692.977578, qy_avg = 43.494311
output_4.tec: qx_avg = -862.199877, qy_avg = -132.759848
output_5.tec: qx_avg = -1478.664791, qy_avg = -828.644369
output_6.tec: qx_avg = -3911.038159, qy_avg = -3607.608115

Comparing consecutive outputs:
Between output_1.tec and output_2.tec:
  Δqx = -43.354662, relative change = 6.907%
  Δqy = -16.153366, relative change = 17.635%
Between output_2.tec and output_3.tec:
  Δqx = -65.284082, relative change = 9.421%
  Δqy = -48.103861, relative change = 110.598%
Between output_3.tec and output_4.tec:
  Δqx = -169.222299, relative change = 19.627%
  Δqy = -176.254159, relative change = 132.762%
Between output_4.tec and output_5.tec:
  Δqx = -616.464914, relative change = 41.691%
  Δqy = -695.884521, relative change 

In [15]:
import numpy as np
import glob

def extract_temperature(filename):
    variables, data = read_tec(filename)
    try:
        temp_idx = variables.index("T")
    except ValueError:
        raise ValueError("Temperature variable not found in variables list.")
    temperature = data[:, temp_idx]
    return temperature

def relative_error(val_old, val_new):
    return abs(val_new - val_old) / (abs(val_new) + 1e-15)  # avoid divide by zero

def main():
    files = sorted(glob.glob('output_*.tec'))
    print(f"Found files: {files}")

    max_temps = []
    min_temps = []

    for f in files:
        temp = extract_temperature(f)
        max_temps.append(np.max(temp))
        min_temps.append(np.min(temp))

    print("\nRelative errors between consecutive files:")
    for i in range(len(files) - 1):
        rel_err_max = relative_error(max_temps[i], max_temps[i+1])
        rel_err_min = relative_error(min_temps[i], min_temps[i+1])
        print(f"Between {files[i]} and {files[i+1]}:")
        print(f"  Max temp relative error: {rel_err_max*100:.4f}% (from {max_temps[i]:.6f} to {max_temps[i+1]:.6f})")
        print(f"  Min temp relative error: {rel_err_min*100:.4f}% (from {min_temps[i]:.6f} to {min_temps[i+1]:.6f})")

if __name__ == "__main__":
    main()


Found files: ['output_1.tec', 'output_2.tec', 'output_3.tec', 'output_4.tec', 'output_5.tec', 'output_6.tec']

Relative errors between consecutive files:
Between output_1.tec and output_2.tec:
  Max temp relative error: 0.3719% (from 17.500000 to 17.565330)
  Min temp relative error: 15.7406% (from -20.989000 to -24.909990)
Between output_2.tec and output_3.tec:
  Max temp relative error: 0.0764% (from 17.565330 to 17.578760)
  Min temp relative error: 14.7071% (from -24.909990 to -29.205240)
Between output_3.tec and output_4.tec:
  Max temp relative error: 0.0271% (from 17.578760 to 17.583530)
  Min temp relative error: 26.9466% (from -29.205240 to -39.977920)
Between output_4.tec and output_5.tec:
  Max temp relative error: 0.0028% (from 17.583530 to 17.584020)
  Min temp relative error: 49.6932% (from -39.977920 to -79.468250)
Between output_5.tec and output_6.tec:
  Max temp relative error: 0.0027% (from 17.584020 to 17.584490)
  Min temp relative error: 66.1676% (from -79.468250 t